# GEE TOOLS
## 6 - Mapa de iluminación
Extrae los datos del archivo de resultados de mapas de iluminación de EnergyPlus&reg;. Este notebook contiene inicialmente el código a implementar para leer estos archivos, después se utilizará para mostrar su funcionamiento con ejemplos.

In [1]:
import pandas as pd
from datetime import datetime

Parámetros a utilizar en la función o clase que se creará en el código para leer los archivos

In [2]:
# Parámetros
file_name = "../data/eplusmap.csv"
date_format = " %m/%d %H:%M"
year = 2000

#### Función is_date()
Si la línea tiene fecha lanza _True_ de lo contrario, lanza _False_

In [3]:
def is_date(text:str, date_format=date_format):
    try:
        datetime.strptime(text, ' %m/%d %H:%M')
        return True
    except:
        return False

### Función get_info()
Si la línea tiene los datos de información del illumination map los extrae como un tupple ordenado como nombre, tz y RefPts

In [4]:
def get_info(text:str):
    name = text.split(',')[1].split(':')[2].split(" at ")[0]
    tz = text.split(',')[1].split(':')[0]
    RefPts = [float(x) for x in text.split(',')[2].split('(')[1].split(')')[0].split(':')]
    return name, tz, RefPts 

Extraemos la información del archivo y lo pasamos a una lista llamada _illum_map_

In [5]:
with open(file_name, "r") as file:
    illum_map = file.read().splitlines()

Ahora se creará una lista llamada _illumaps_ con todos diccionarios que tienen todos los mapas de iluminación del archivo.

In [44]:
illumaps = []
initial = True # initial se usa para que en el primer bucle no guarde ningún diccionario.
for i in range(len(illum_map)):
    X_value = illum_map[i].split(',')[0] # Evaluamos el texto antes de la coma del archivo
    if (X_value == "Date/Time"): # Si el valor comienza con "Date/Time es de información y comienzo de un mapa"
        # Tomar línea de información y guardar datos contenidos utilizando función get_info
        info = get_info(illum_map[i]) 
    elif is_date(X_value): #Si la línea tiene fecha, contiene la hora del mapa y los datos de coordenadas en Y
        if not initial: # Si no es el primer bucle tomará los datos recopilados y los guardará como un diccionario
            illudata.index = Y_array #Cambia el nombre de las hileras
            illumap["Data"] = illudata.transpose() # Transpone los datos del DataFrame para que Y sea las columnas y X las hileras
            illumaps.append(illumap) # Guarda el diccionario en illumap
        else:
            initial = False # No guarda el diccionario pero se pasa a False para que lo cree en el próximo paso por línea de información
        # Preparar diccionario y DataFrame para guardar información extraida
        illumap = {}
        illudata = pd.DataFrame()
        illumap["Timestamp"] = datetime.strptime(X_value,' %m/%d %H:%M').replace(year=year)
        illumap["Name"] = info[0]
        illumap["Thermal zone"] = info[1]
        illumap["Ref points"] = info[2]
        Y_array = [y[:-1] for y in illum_map[i].split(',')[1:]]
    else: # Las otras líneas contienen la información de iluminación para cada coordenada en X la cual se guarda en el DataFrame
        illudata[X_value[:-1]] = [float(x) for x in illum_map[i].split(',')[1:]]

In [46]:
illumaps[0]["Data"]

,(-2.28E-015;9.95E-016),(0.29;0.42),(0.57;0.85),(0.86;1.27),(1.15;1.70),(1.43;2.12),(1.72;2.55),(2.00;2.97),(2.29;3.40),(2.58;3.82),(2.86;4.25),(3.15;4.67),(3.44;5.09),(3.72;5.52),(4.01;5.94),(4.30;6.37),(4.58;6.79),(4.87;7.22),(5.15;7.64),(5.44;8.07)
(-2.28E-015;9.95E-016),175.0,356.0,355.0,355.0,353.0,1088.0,1196.0,1202.0,1196.0,1188.0,1176.0,1145.0,1135.0,1172.0,1188.0,1199.0,1200.0,1200.0,1177.0,175.0
(-0.40;0.27),175.0,852.0,870.0,839.0,823.0,908.0,1060.0,1087.0,1071.0,820.0,897.0,1039.0,1048.0,1069.0,1022.0,1027.0,1112.0,1097.0,946.0,175.0
(-0.79;0.53),175.0,752.0,825.0,822.0,846.0,872.0,959.0,991.0,969.0,827.0,877.0,961.0,991.0,1001.0,960.0,1004.0,996.0,948.0,754.0,175.0
(-1.19;0.80),175.0,683.0,747.0,793.0,919.0,810.0,861.0,893.0,862.0,813.0,828.0,892.0,911.0,912.0,910.0,913.0,895.0,824.0,650.0,175.0
(-1.58;1.07),175.0,669.0,751.0,756.0,697.0,752.0,775.0,794.0,763.0,733.0,784.0,803.0,812.0,830.0,839.0,809.0,788.0,728.0,585.0,175.0
(-1.98;1.33),175.0,651.0,701.0,798.0,659.0,695.0,708.0,705.0,715.0,714.0,727.0,724.0,737.0,759.0,783.0,747.0,691.0,630.0,531.0,175.0
(-2.37;1.60),175.0,629.0,672.0,749.0,606.0,648.0,647.0,656.0,681.0,689.0,668.0,681.0,708.0,717.0,718.0,697.0,656.0,569.0,482.0,175.0
(-2.77;1.87),175.0,592.0,672.0,732.0,613.0,624.0,632.0,664.0,667.0,647.0,681.0,685.0,690.0,691.0,649.0,643.0,601.0,521.0,456.0,175.0
(-3.16;2.13),175.0,583.0,658.0,684.0,574.0,593.0,598.0,607.0,607.0,625.0,635.0,640.0,642.0,592.0,583.0,591.0,550.0,489.0,415.0,175.0
(-3.56;2.40),175.0,566.0,624.0,650.0,539.0,554.0,558.0,569.0,570.0,586.0,588.0,600.0,600.0,587.0,580.0,546.0,508.0,456.0,400.0,175.0
